In [74]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import holidays
pd.set_option('display.max_rows', 4)

In [75]:
df = pd.read_csv('Denver.csv')

In [76]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [77]:
# Convert string to datetime for all dates
datetimes = []
for i in range(df.shape[0]):
    datetimes.append(datetime.strptime(df.iloc[i]['FIRST_OCCURRENCE_DATE'], '%m/%d/%Y %I:%M:%S %p'))

In [78]:
# Using the datetime column to create the year, month... columns
year = []
month = []
day = []
day_of_week = []
hour = []
minute = []
for date in datetimes:
    year.append(date.year)
    month.append(date.month)
    day.append(date.day)
    hour.append(date.hour)
    minute.append(date.minute)
    day_of_week.append(date.weekday())

In [79]:
# Creating the columns
date_df = pd.DataFrame(list())
date_df["Date-key"] = df.index
date_df["Date-key"] += 193478
date_df["Year"] = year
date_df["Month"] = month
date_df["Day"] = day
date_df["Hour"] = hour
date_df["Minute"] = minute
date_df["Day-of-week"] = day_of_week

In [80]:
# Create the weekend column
weekend = []
for i in day_of_week:
    if i < 5:
        weekend.append(False)
    else:
        weekend.append(True)
date_df["Weekend"] = weekend

In [81]:
# Maps te days of the week to their correspondent string name
date_df_map = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 
               3: "Thursday", 4: "Friday", 5: "Saturday",
               6: "Sunday"}
date_df = date_df.replace({"Day-of-week": date_df_map})

In [82]:
# Check for holidays and creates the holiday column
isHoliday = []
ca_holidays = holidays.CA() 
for i in datetimes:
    isHoliday.append(i in ca_holidays)
date_df["Holiday"] = isHoliday

In [83]:
# Get the holiday names and create holiday name column
holiday_name = []
for i in datetimes:
    holiday_name.append(ca_holidays.get(i))
date_df["Holiday-name"] = holiday_name

In [84]:
# Reorganize the columns
cols = ["Date-key", "Day-of-week", "Minute","Hour", "Day", "Month"
       , "Year", "Weekend", "Holiday", "Holiday-name"]
date_df = date_df[cols]

In [85]:
date_df

,Date-key,Day-of-week,Minute,Hour,Day,Month,Year,Weekend,Holiday,Holiday-name
0,193478,Wednesday,31,23,15,6,2016,False,False,None
1,193479,Wednesday,30,12,11,10,2017,False,False,None
...,...,...,...,...,...,...,...,...,...,...
462403,655881,Saturday,20,17,16,11,2019,True,False,None
462404,655882,Friday,45,6,27,12,2019,False,False,None


In [86]:
date_df.to_csv("Denver_Date_Final.csv", index = False)